# Reporte de producción mensual

In [1]:
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def format_timedelta_hh_mm_ss(td: timedelta) -> str:
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

config = {
  'user': 'operario',
  'password': '',
  'host': '192.168.0.12', # O la IP de tu servidor
  'database': 'dbp8100'
}

def conectar_mysql():
    return mysql.connector.connect(**config)
    
def obtener_df(fecha_inicio, fecha_fin):
    db = conectar_mysql()
    cur = db.cursor()
    cur.execute('''
                SELECT
                    subconsulta.Codigo,
                    subconsulta.Nombre,
                    SUM(subconsulta.Dosificado) AS Total
                FROM
                    (   SELECT
                            tareaseje.NroID,
                            tareaseje.Fecha AS Fecha,
                            tareaseje.Hora AS Hora,
                            formulas.NroID AS IDF,
                            formulas.Codigo AS Codigo,
                            formulas.Nombre AS Nombre,
                            tareaseje.Set AS Programado,
                            (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                            tareaseje.Tiempo AS Tiempo
                        FROM
                            dbp8100.tareaseje AS tareaseje
                        JOIN
                            dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
                        WHERE
                            (tareaseje.Fecha = %s AND tareaseje.Hora >= '22:00:00') OR
                            (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s AND tareaseje.Hora < '22:00:00')
                    ) AS subconsulta
                -- Agrupamos por el nombre de la fórmula para sumar los valores dosificados
                GROUP BY
                    subconsulta.Nombre;
                ''',(fecha_inicio, fecha_inicio, fecha_fin))

    resultados = cur.fetchall()
    # Obtener los nombres de las columnas
    columnas = [desc[0] for desc in cur.description]

    # Crear un DataFrame de pandas
    df = pd.DataFrame(resultados, columns=columnas)

    return df

In [5]:
df = obtener_df('2025-08-31', '2025-09-30')
df

,Codigo,Nombre,Total
0,FP0027,(LA MALVINA) DESARROLLO TERNEROS MAIZ SIN SAL,6038.10
1,FP0009,(LA MALVINA) TERNEROS DESTETE,12121.10
2,51-004,CERDAS GESTACIÓN (MAXIMIX),999.60
3,FC820,CERDAS GESTACIÓN (LA MALVINA) MAXIMIX.,6009.20
4,FC913,CERDAS LACTANCIA (MAXIMIX) LA MALVINA..,3988.90
5,09-141,CRE. MELAZA 10% A.A. (Los Roanos),179602.95
6,09-142,CREC. MELAZA 12 % A.A...,215328.95
7,09-143,CREC. MELAZA 12 % A.A....,316544.75
8,FC633,DESARROLLO (LA MALVINA)...,30060.60
9,DPO1,DESARROLLO PORCINO AP expeller,5972.80


In [6]:
df.to_excel('09.xlsx', index=False)